# Jump process using ModelingToolkit.jl
Simon Frost (@sdwfrost), 2022-03-17

## Introduction

This implementation defines the model as a combination of two jump processes, infection and recovery, simulated using the [Doob-Gillespie algorithm](https://en.wikipedia.org/wiki/Gillespie_algorithm). This example follows closely with the example in the [ModelingToolkit.jl documentation](https://mtk.sciml.ai/dev/systems/JumpSystem/#ModelingToolkit.JumpSystem).

## Libraries

In [ ]:
using ModelingToolkit
using DiffEqJump
using Random
using DataFrames
using Tables
using StatsPlots
using BenchmarkTools

## Transitions

For each process, we define the rate at which it occurs, and how the state variables change at each jump. Note that these are total rates, not *per capita*, and that the change in state variables occurs in-place.

In [ ]:
@parameters β c γ
@variables t S(t) I(t) R(t)
N = S+I+R
rate₁   = β*c*S*I/N
affect₁ = [S ~ S - 1, I ~ I + 1]
rate₂   = γ*I
affect₂ = [I ~ I - 1, R ~ R + 1]
j₁      = ConstantRateJump(rate₁,affect₁)
j₂      = ConstantRateJump(rate₂,affect₂)
@named sir_js = JumpSystem([j₁,j₂], t, [S,I,R], [β,c,γ])

## Time domain

In [ ]:
tmax = 40.0
tspan = (0.0,tmax)
δt = 1.0;

## Initial conditions

In ModelingToolkit.jl, initial conditions (as well as parameters) are given as dictionaries.

In [ ]:
u0 = [S => 990, I => 10, R => 0];

## Parameter values

In [ ]:
p = [β => 0.05, c => 10.0, γ => 0.25];

## Random number seed

We set a random number seed for reproducibility.

In [ ]:
Random.seed!(1234);

## Running the model

Running this model involves:

- Turning the `JumpSystem` into a `DiscreteProblem`;
- Combining the `JumpSystem` and the `DiscreteProblem`  into a `JumpProblem`; and
- Solving the `JumpProblem`, specifying `SSAStepper()`

In [ ]:
sir_dprob = DiscreteProblem(sir_js, u0, tspan,p);

In [ ]:
sir_jprob = JumpProblem(sir_js, sir_dprob,Direct());

In [ ]:
sol_jump = solve(sir_jprob,SSAStepper());

## Post-processing

In order to get output comparable across implementations, we output the model at a fixed set of times.

In [ ]:
out_jump = sol_jump(0:δt:tmax);

We can convert to a dataframe for convenience.

In [ ]:
df_jump = DataFrame(Tables.table(out_jump'))
rename!(df_jump,["S","I","R"])
df_jump[!,:t] = out_jump.t;

## Plotting

We can now plot the results.

In [ ]:
@df df_jump plot(:t,
    [:S :I :R],
    xlabel="Time",
    ylabel="Number")

## Benchmarking

In [ ]:
@benchmark solve(sir_jprob,SSAStepper())